# Collaborative Filtering - Movie Recommendation Project
---

## BUSINESS PROBLEM
---
The online movie streaming platform (for example, superperfectmoviesblabla.tv) wants to develop a recommendation system using collaborative filtering. They aim to generate recommendations that encompass the opinions of the company community who have experimented with content-based recommendation systems. When users like a movie, they want to be recommended other movies that have similar likability patterns.

## DATASET STORY
---

The dataset has been provided by MovieLens, containing movies and the ratings given to these movies. The dataset includes approximately 2,000,000 ratings for around 27,000 movies.

## FEATURES
---
## movie.csv
- **movieId:** Unique movie number (UniqueID)
- **title:** Movie title

## rating.csv
- **userId:** Unique user number (UniqueID)
- **movieId:** Unique movie number (UniqueID)
- **rating:** Rating given to the movie by the user
- **timestamp:** Rating date

## STEPS
---
### Step 1: Data Preprocessing
### Step 2: Creating User-Movie DataFrame
### Step 3: Making Item-Based Movie Recommendations
### Step 4: Making User-Based Movie Recommendations
### Step 5: Model Based Matrix Factorization
### Step 6: Preparation of Working Script
---

### *Libraries and Settings*
---

In [1]:
############################################################################################################################################
import pandas as pd
import numpy as np
import warnings
############################################################################################################################################
pd.set_option("display.max_columns", 500)
warnings.filterwarnings("ignore")
############################################################################################################################################

---
# Data Preprocessing
---

In [ ]:
############################################################################################################################################
movies = pd.read_csv("data/movies.csv")
ratings = pd.read_csv("data/ratings.csv")
############################################################################################################################################
df = ratings.merge(movies, how="left")
df.head()
############################################################################################################################################

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War
4,1,899,3.5,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance


In [3]:
df.shape

(25000095, 6)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
 4   title      object 
 5   genres     object 
dtypes: float64(1), int64(3), object(2)
memory usage: 1.1+ GB


In [5]:
# convert timestamp to datetime dtype
df["timestamp"] = pd.to_datetime(df["timestamp"], unit= "s").dt.date
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,2006-05-17,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,306,3.5,2006-05-17,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama
2,1,307,5.0,2006-05-17,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,1,665,5.0,2006-05-17,Underground (1995),Comedy|Drama|War
4,1,899,3.5,2006-05-17,Singin' in the Rain (1952),Comedy|Musical|Romance


In [6]:
# check for missing values
df.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
title        0
genres       0
dtype: int64

It is important to use memory efficiently while working with matrix structures in recommendation systems which is required a 'User-Movies Matrix' for correlation calculation in the next step.

That's why we define a threshold to eliminate the movies rated at least 1000 times by users and the users rated at least 30 movies, which we think that might not reflect a collaboration with other users and their preferences.

In [7]:
############################################################################################################################################
# count of movies rated by users based on threshold of 1000
title_count = pd.DataFrame(df["movieId"].value_counts(), columns=["count"])
m_less_than_1000 = len(title_count[title_count["count"] < 1000])
m_more_than_1000 = len(title_count[title_count["count"] >= 1000])

print("*"*100)
print("Total Movie Count: ", len(title_count))
print("*"*100)
print("Movie count rated less than 1000 times: ", m_less_than_1000,"(", round((m_less_than_1000 / len(title_count))*100,2), " %)")
print("Movie count rated more than 1000 times: ", m_more_than_1000,"(", round((m_more_than_1000 / len(title_count))*100,2), " %)")
print("*"*100)

############################################################################################################################################
# count of users rated movies based on threshold of 30
user_count = pd.DataFrame(df["userId"].value_counts(), columns=["count"])
u_less_than_30 = len(user_count[user_count["count"] < 30])
u_more_than_30 = len(user_count[user_count["count"] >= 30])

print("*"*100)
print("Total User Count: ", len(user_count))
print("*"*100)
print("User count rated less than 30 times: ", u_less_than_30,"(", round((u_less_than_30 / len(user_count))*100,2), " %)")
print("User count rated more than 30 times: ", u_more_than_30,"(", round((u_more_than_30 / len(user_count))*100,2), " %)")
print("*"*100)
############################################################################################################################################

****************************************************************************************************
Total Movie Count:  59047
****************************************************************************************************
Movie count rated less than 1000 times:  55253 ( 93.57  %)
Movie count rated more than 1000 times:  3794 ( 6.43  %)
****************************************************************************************************
****************************************************************************************************
Total User Count:  162541
****************************************************************************************************
User count rated less than 30 times:  28650 ( 17.63  %)
User count rated more than 30 times:  133891 ( 82.37  %)
****************************************************************************************************


6.43% of movies are rated less than 1000 and 17.63% users rated at least 30 movies.
We remove them to reduce matrix size for efficiency while keeping most of the information in dataset. 

In [8]:
movies_to_be_removed = title_count[title_count["count"] < 1000].index.to_list()
users_to_be_removed = user_count[user_count["count"] < 30].index.to_list()

rec_df = df[~df["movieId"].isin(movies_to_be_removed)]
rec_df = rec_df[~rec_df["userId"].isin(users_to_be_removed)]

display(rec_df)

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,2006-05-17,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,306,3.5,2006-05-17,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama
2,1,307,5.0,2006-05-17,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,1,665,5.0,2006-05-17,Underground (1995),Comedy|Drama|War
4,1,899,3.5,2006-05-17,Singin' in the Rain (1952),Comedy|Musical|Romance
...,...,...,...,...,...,...
25000090,162541,50872,4.5,2009-04-28,Ratatouille (2007),Animation|Children|Drama
25000091,162541,55768,2.5,2009-04-28,Bee Movie (2007),Animation|Comedy
25000092,162541,56176,2.0,2009-04-28,Alvin and the Chipmunks (2007),Children|Comedy
25000093,162541,58559,4.0,2009-04-28,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX


Now, 21.48 Million records are kept out of 25 Million although we reduced column count 93.56% and 82.37% of row count.

In [9]:
before = len(user_count) * len(title_count)
after = len(rec_df["userId"].value_counts()) * len(rec_df["movieId"].value_counts())

print("Matrix Size BEFORE reduction:", len(user_count), "*", len(title_count),"=", before)
print("Matrix Size AFTER reduction:", len(rec_df["userId"].value_counts()), "*", len(rec_df["movieId"].value_counts()),"=", after)
print("Matrix Size Reduction Rate:", round((before - after)/before, 2)*100, "%")
print("Information kept:", round(len(rec_df)/len(df), 2)*100, "%")

Matrix Size BEFORE reduction: 162541 * 59047 = 9597558427
Matrix Size AFTER reduction: 133891 * 3794 = 507982454
Matrix Size Reduction Rate: 95.0 %
Information kept: 86.0 %


In other words, we reduced matrix size by 95% while keeping 86% of information.

---
# Creating User-Movie DataFrame
---

Thus, we can create 'User-Movies Matrix' using pivot table.

In [10]:
user_movie_matrix = rec_df.pivot_table(index="userId", columns="movieId", values="rating")
user_movie_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,39,41,42,43,44,45,46,47,48,50,52,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,76,78,79,81,82,85,86,88,89,92,93,94,95,97,100,101,102,103,104,105,107,110,111,112,117,118,122,123,125,126,132,135,140,141,144,145,147,149,150,151,153,154,155,156,157,158,159,160,161,162,163,164,165,168,169,170,171,172,173,174,175,176,177,179,180,181,185,186,188,191,193,194,195,196,198,199,203,204,205,207,208,213,215,216,218,222,223,224,225,227,229,230,231,232,233,234,235,236,237,239,242,246,247,248,249,252,253,256,257,258,259,260,261,262,265,266,267,270,271,272,273,274,275,276,277,278,280,281,282,288,289,290,292,293,296,299,300,302,303,305,306,307,308,312,313,314,315,316,317,318,319,322,326,327,328,329,330,332,333,334,337,338,339,340,342,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,360,361,362,364,365,366,367,368,...,107141,107348,107406,107771,108188,108190,108727,108729,108932,108945,109374,109487,109578,109673,109848,110102,110127,110501,110553,110730,110882,111113,111360,111362,111364,111443,111622,111659,111743,111759,111781,111921,112138,112171,112175,112183,112290,112370,112552,112556,112623,112804,112852,112897,113345,113348,113378,113573,113741,114180,114662,114795,114935,115122,115149,115210,115569,115617,115713,116161,116797,116799,116823,116897,116977,117176,117444,117529,117533,117590,117881,117895,118696,118900,118997,119141,119145,119155,120466,120635,120799,121231,122882,122886,122890,122892,122896,122898,122900,122902,122904,122906,122910,122912,122914,122916,122918,122920,122922,122924,122926,125916,127098,127108,127202,128360,128838,129354,130073,130490,130520,130634,132046,132480,132796,133419,133771,134130,134170,134368,134393,134853,135133,135137,135143,135436,135536,135567,135569,135861,135887,136018,136020,136562,136864,137857,138036,139385,139644,140110,140174,140247,140267,140715,140956,141866,142448,142488,142997,143355,143385,143859,146656,148626,149334,149406,152017,152077,152081,152970,156387,157296,157699,158238,158783,158872,158966,159093,159415,159817,159858,160080,160271,160438,160718,160980,161131,161582,161634,162082,162350,162414,162578,162600,162606,163134,163645,164179,164909,165549,165551,166024,166461,166528,166534,166635,166643,166946,167746,168248,168250,168252,168254,168326,168366,168492,168612,169864,169984,170705,171011,171763,173145,173291,173941,174055,175303,175569,175661,176101,176371,176419,177593,177615,177763,177765,178061,179401,179817,179819,180031,182715,183611,183837,183869,183897,185029,187541,187593,187595,188301,189203,189333,189713,192385,192389,192803,194448,195159,201773
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.5,5

Let's select a movieId and get first 10 movies correlated with that movie which are similarly rated movieIds with selected movieId. 

In [11]:
# let's take movie with Id 8 as example
selected_movieId = user_movie_matrix.loc[:,8]
correlated_movieIds = user_movie_matrix.corrwith(selected_movieId).sort_values(ascending = False)[:10]
correlated_movieIds

movieId
8         1.000000
90522     0.974316
189203    0.965535
89118     0.919393
69524     0.869783
74228     0.859885
112804    0.856975
162082    0.852198
52694     0.848627
95105     0.834364
dtype: float64

---
# Making Item-Based Movie Recommendations
---

First, we define some function to make searching easier for us.

In [12]:
def find_movie_title_using_id(movieId):
    return rec_df[rec_df["movieId"] == movieId]["title"].values[0]

def find_movie_id_using_name(movie_name):
    return rec_df[rec_df["title"] == movie_name]["movieId"].values[0]

def find_movie_names_with_keyword(seached_movie_name):
    movie_names = list()
    [movie_names.append(name) for name in rec_df["title"] if seached_movie_name.lower() in name.lower() and name not in movie_names]
    return movie_names

Now, we seach for a movie using a key.

In [13]:
seached_key_word = "legend"
movie_full_name = find_movie_names_with_keyword(seached_key_word)
movie_full_name

['Legends of the Fall (1994)',
 'Anchorman: The Legend of Ron Burgundy (2004)',
 'Legend of Zorro, The (2005)',
 'I Am Legend (2007)',
 'Anchorman 2: The Legend Continues (2013)',
 "City Slickers II: The Legend of Curly's Gold (1994)",
 'Urban Legend (1998)',
 'Legend of Drunken Master, The (Jui kuen II) (1994)',
 'Urban Legends: Final Cut (2000)',
 'Legend (1985)',
 'Legend of Bagger Vance, The (2000)',
 "Legend of the Guardians: The Owls of Ga'Hoole (2010)"]

As we have found the full name of the movie (assuming that we seached for 'I Am Legend (2007)'), we get related movieId using defined function.

In [14]:
searched_movie_id = find_movie_id_using_name(movie_full_name[3])
searched_movie_id

56174

After getting movieIds correlated with selected movie, we can find first 10 recommendations.

In [15]:
selected_movieId = user_movie_matrix.loc[:,searched_movie_id]
correlated_movies = user_movie_matrix.corrwith(selected_movieId).sort_values(ascending = False)[1:11] # [1:11] defined as movie itself is in index 0
correlated_movieIds = correlated_movies.index.to_list()
correlated_rate = correlated_movies.to_list()

print("*"*100)
print("'", movie_full_name[3], "' - Recommendation List")
print("*"*100)
for no, movieId, rate in zip(range(1,11),correlated_movieIds, correlated_rate):
    movie_name = find_movie_title_using_id(movieId)
    print(no,"-", movie_name, "-"*(70 - len(movie_name)-len(str(no))), " - ", round(rate*100,2),"%")
print("*"*100)

****************************************************************************************************
' I Am Legend (2007) ' - Recommendation List
****************************************************************************************************
1 - Widows' Peak (1994) --------------------------------------------------  -  68.45 %
2 - I, Robot (2004) ------------------------------------------------------  -  61.91 %
3 - Hancock (2008) -------------------------------------------------------  -  55.55 %
4 - World War Z (2013) ---------------------------------------------------  -  52.76 %
5 - Book of Eli, The (2010) ----------------------------------------------  -  52.02 %
6 - Preacher's Wife, The (1996) ------------------------------------------  -  51.1 %
7 - Shanghai Triad (Yao a yao yao dao waipo qiao) (1995) -----------------  -  49.21 %
8 - Focus (2015) ---------------------------------------------------------  -  48.48 %
9 - Day After Tomorrow, The (2004) -----------------------

---
# Making User-Based Movie Recommendations
---

The purpose of this part is to find the movies that is most probable liked by a selected user.

Let's first remember 'User-Movie Matrix' created previously.

In [16]:
user_movie_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,39,41,42,43,44,45,46,47,48,50,52,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,76,78,79,81,82,85,86,88,89,92,93,94,95,97,100,101,102,103,104,105,107,110,111,112,117,118,122,123,125,126,132,135,140,141,144,145,147,149,150,151,153,154,155,156,157,158,159,160,161,162,163,164,165,168,169,170,171,172,173,174,175,176,177,179,180,181,185,186,188,191,193,194,195,196,198,199,203,204,205,207,208,213,215,216,218,222,223,224,225,227,229,230,231,232,233,234,235,236,237,239,242,246,247,248,249,252,253,256,257,258,259,260,261,262,265,266,267,270,271,272,273,274,275,276,277,278,280,281,282,288,289,290,292,293,296,299,300,302,303,305,306,307,308,312,313,314,315,316,317,318,319,322,326,327,328,329,330,332,333,334,337,338,339,340,342,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,360,361,362,364,365,366,367,368,...,107141,107348,107406,107771,108188,108190,108727,108729,108932,108945,109374,109487,109578,109673,109848,110102,110127,110501,110553,110730,110882,111113,111360,111362,111364,111443,111622,111659,111743,111759,111781,111921,112138,112171,112175,112183,112290,112370,112552,112556,112623,112804,112852,112897,113345,113348,113378,113573,113741,114180,114662,114795,114935,115122,115149,115210,115569,115617,115713,116161,116797,116799,116823,116897,116977,117176,117444,117529,117533,117590,117881,117895,118696,118900,118997,119141,119145,119155,120466,120635,120799,121231,122882,122886,122890,122892,122896,122898,122900,122902,122904,122906,122910,122912,122914,122916,122918,122920,122922,122924,122926,125916,127098,127108,127202,128360,128838,129354,130073,130490,130520,130634,132046,132480,132796,133419,133771,134130,134170,134368,134393,134853,135133,135137,135143,135436,135536,135567,135569,135861,135887,136018,136020,136562,136864,137857,138036,139385,139644,140110,140174,140247,140267,140715,140956,141866,142448,142488,142997,143355,143385,143859,146656,148626,149334,149406,152017,152077,152081,152970,156387,157296,157699,158238,158783,158872,158966,159093,159415,159817,159858,160080,160271,160438,160718,160980,161131,161582,161634,162082,162350,162414,162578,162600,162606,163134,163645,164179,164909,165549,165551,166024,166461,166528,166534,166635,166643,166946,167746,168248,168250,168252,168254,168326,168366,168492,168612,169864,169984,170705,171011,171763,173145,173291,173941,174055,175303,175569,175661,176101,176371,176419,177593,177615,177763,177765,178061,179401,179817,179819,180031,182715,183611,183837,183869,183897,185029,187541,187593,187595,188301,189203,189333,189713,192385,192389,192803,194448,195159,201773
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.5,5

We select a random user and find the movieIds that he/she rated.

In [17]:
selected_userId = user_movie_matrix.sample(1).index[0]
#158867
print("*"*100)
print("Selected UserId:", selected_userId)
print("*"*100)

umm_for_selected_userId = user_movie_matrix[user_movie_matrix.index == selected_userId] # 'umm' is used as abbreviation of 'user_movie_matrix'
umm_for_selected_userId

****************************************************************************************************
Selected UserId: 63131
****************************************************************************************************


movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,39,41,42,43,44,45,46,47,48,50,52,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,76,78,79,81,82,85,86,88,89,92,93,94,95,97,100,101,102,103,104,105,107,110,111,112,117,118,122,123,125,126,132,135,140,141,144,145,147,149,150,151,153,154,155,156,157,158,159,160,161,162,163,164,165,168,169,170,171,172,173,174,175,176,177,179,180,181,185,186,188,191,193,194,195,196,198,199,203,204,205,207,208,213,215,216,218,222,223,224,225,227,229,230,231,232,233,234,235,236,237,239,242,246,247,248,249,252,253,256,257,258,259,260,261,262,265,266,267,270,271,272,273,274,275,276,277,278,280,281,282,288,289,290,292,293,296,299,300,302,303,305,306,307,308,312,313,314,315,316,317,318,319,322,326,327,328,329,330,332,333,334,337,338,339,340,342,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,360,361,362,364,365,366,367,368,...,107141,107348,107406,107771,108188,108190,108727,108729,108932,108945,109374,109487,109578,109673,109848,110102,110127,110501,110553,110730,110882,111113,111360,111362,111364,111443,111622,111659,111743,111759,111781,111921,112138,112171,112175,112183,112290,112370,112552,112556,112623,112804,112852,112897,113345,113348,113378,113573,113741,114180,114662,114795,114935,115122,115149,115210,115569,115617,115713,116161,116797,116799,116823,116897,116977,117176,117444,117529,117533,117590,117881,117895,118696,118900,118997,119141,119145,119155,120466,120635,120799,121231,122882,122886,122890,122892,122896,122898,122900,122902,122904,122906,122910,122912,122914,122916,122918,122920,122922,122924,122926,125916,127098,127108,127202,128360,128838,129354,130073,130490,130520,130634,132046,132480,132796,133419,133771,134130,134170,134368,134393,134853,135133,135137,135143,135436,135536,135567,135569,135861,135887,136018,136020,136562,136864,137857,138036,139385,139644,140110,140174,140247,140267,140715,140956,141866,142448,142488,142997,143355,143385,143859,146656,148626,149334,149406,152017,152077,152081,152970,156387,157296,157699,158238,158783,158872,158966,159093,159415,159817,159858,160080,160271,160438,160718,160980,161131,161582,161634,162082,162350,162414,162578,162600,162606,163134,163645,164179,164909,165549,165551,166024,166461,166528,166534,166635,166643,166946,167746,168248,168250,168252,168254,168326,168366,168492,168612,169864,169984,170705,171011,171763,173145,173291,173941,174055,175303,175569,175661,176101,176371,176419,177593,177615,177763,177765,178061,179401,179817,179819,180031,182715,183611,183837,183869,183897,185029,187541,187593,187595,188301,189203,189333,189713,192385,192389,192803,194448,195159,201773
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
63131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,N

Find out the movies rated by selected user.

In [18]:
bool_for_selected_userId = umm_for_selected_userId.apply(lambda col: col.notnull(), axis=1)
bool_for_selected_userId

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,39,41,42,43,44,45,46,47,48,50,52,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,76,78,79,81,82,85,86,88,89,92,93,94,95,97,100,101,102,103,104,105,107,110,111,112,117,118,122,123,125,126,132,135,140,141,144,145,147,149,150,151,153,154,155,156,157,158,159,160,161,162,163,164,165,168,169,170,171,172,173,174,175,176,177,179,180,181,185,186,188,191,193,194,195,196,198,199,203,204,205,207,208,213,215,216,218,222,223,224,225,227,229,230,231,232,233,234,235,236,237,239,242,246,247,248,249,252,253,256,257,258,259,260,261,262,265,266,267,270,271,272,273,274,275,276,277,278,280,281,282,288,289,290,292,293,296,299,300,302,303,305,306,307,308,312,313,314,315,316,317,318,319,322,326,327,328,329,330,332,333,334,337,338,339,340,342,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,360,361,362,364,365,366,367,368,...,107141,107348,107406,107771,108188,108190,108727,108729,108932,108945,109374,109487,109578,109673,109848,110102,110127,110501,110553,110730,110882,111113,111360,111362,111364,111443,111622,111659,111743,111759,111781,111921,112138,112171,112175,112183,112290,112370,112552,112556,112623,112804,112852,112897,113345,113348,113378,113573,113741,114180,114662,114795,114935,115122,115149,115210,115569,115617,115713,116161,116797,116799,116823,116897,116977,117176,117444,117529,117533,117590,117881,117895,118696,118900,118997,119141,119145,119155,120466,120635,120799,121231,122882,122886,122890,122892,122896,122898,122900,122902,122904,122906,122910,122912,122914,122916,122918,122920,122922,122924,122926,125916,127098,127108,127202,128360,128838,129354,130073,130490,130520,130634,132046,132480,132796,133419,133771,134130,134170,134368,134393,134853,135133,135137,135143,135436,135536,135567,135569,135861,135887,136018,136020,136562,136864,137857,138036,139385,139644,140110,140174,140247,140267,140715,140956,141866,142448,142488,142997,143355,143385,143859,146656,148626,149334,149406,152017,152077,152081,152970,156387,157296,157699,158238,158783,158872,158966,159093,159415,159817,159858,160080,160271,160438,160718,160980,161131,161582,161634,162082,162350,162414,162578,162600,162606,163134,163645,164179,164909,165549,165551,166024,166461,166528,166534,166635,166643,166946,167746,168248,168250,168252,168254,168326,168366,168492,168612,169864,169984,170705,171011,171763,173145,173291,173941,174055,175303,175569,175661,176101,176371,176419,177593,177615,177763,177765,178061,179401,179817,179819,180031,182715,183611,183837,183869,183897,185029,187541,187593,187595,188301,189203,189333,189713,192385,192389,192803,194448,195159,201773
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
63131,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fa

In [19]:
movieIds_ratedby_selected_userId = bool_for_selected_userId.iloc[0].loc[lambda movieId: movieId == True]
list_movieIds_ratedby_selected_userId = bool_for_selected_userId.iloc[0].loc[lambda movieId: movieId == True].index.to_list()

print("Movie count rated by UserId '", selected_userId,"' =", len(list_movieIds_ratedby_selected_userId))

Movie count rated by UserId ' 63131 ' = 76


Leave only movieIds rated by selected user considering all users.

In [20]:
allUserIds_and_movies_ratedby_selected_userId = user_movie_matrix.loc[:, list_movieIds_ratedby_selected_userId]
allUserIds_and_movies_ratedby_selected_userId # 180 columns, the same with selected userId 

movieId,16,32,111,296,508,593,903,914,915,916,1089,1193,1213,1228,1246,1247,1302,1387,1466,1580,1954,1968,2067,2324,2329,2618,2858,2997,3252,3408,3552,3949,4011,4022,4034,4226,4262,4306,4448,4776,4963,4973,4993,4995,5218,5445,5989,6016,6807,6874,7143,7254,7361,7438,8464,8596,8622,8981,30707,30812,32587,33166,48516,49272,53894,57669,58559,68157,69481,72998,74458,79132,81562,99114,106782,117533
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,3.0,NaN,NaN,4.5,NaN,1.0,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,4.5,NaN,NaN,4.0,NaN,5.0,5.0,NaN,3.5,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,4.5,4.0,5.0,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,5.0,NaN,NaN,3.5,NaN,NaN,4.0,4.0,4.0,4.0,4.5,4.0,4.0,4.0,4.0,NaN,4.0,4.0,4.0,3.5,4.0,4.0,NaN,4.5,2.5,NaN,4.5,4.0,NaN,NaN,NaN,NaN,4.0,NaN,4.0,4.0,5.0,4.0,NaN,3.0,5.0,4.0,3.5,4.0,4.0,5.0,NaN,4.0,4.5,4.0
4,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,4.5,3.5,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,4.5,4.5,3.5,NaN,2.5,5.0,4.5,NaN,4.0,NaN,NaN
5,NaN,5.0,NaN,4.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162537,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162538,NaN,NaN,4.0,2.5,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,3.5,NaN,3.0,NaN,NaN,NaN,4.5,NaN,4.0,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,4.0,4.0,3.0,NaN,4.5,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,2.0,3.0,4.0,NaN,4.0,NaN,NaN
162539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Exclude users who have rated movies that were also rated by the selected user by at least the chosen percentage.

In [21]:
perc_threshold_rated_same_movies = 0.7 # the threshold for users who rated at least 70% of the same movies rated by selected user
count_threshold_rated_same_movies = len(list_movieIds_ratedby_selected_userId) * perc_threshold_rated_same_movies
userIds_rated_atleast_X_perc_same_movies_with_selected_userId = allUserIds_and_movies_ratedby_selected_userId[allUserIds_and_movies_ratedby_selected_userId.notnull().sum(axis=1) > count_threshold_rated_same_movies]
userIds_rated_atleast_X_perc_same_movies_with_selected_userId

movieId,16,32,111,296,508,593,903,914,915,916,1089,1193,1213,1228,1246,1247,1302,1387,1466,1580,1954,1968,2067,2324,2329,2618,2858,2997,3252,3408,3552,3949,4011,4022,4034,4226,4262,4306,4448,4776,4963,4973,4993,4995,5218,5445,5989,6016,6807,6874,7143,7254,7361,7438,8464,8596,8622,8981,30707,30812,32587,33166,48516,49272,53894,57669,58559,68157,69481,72998,74458,79132,81562,99114,106782,117533
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
426,3.5,3.0,4.5,4.5,3.5,5.0,4.0,NaN,4.0,NaN,4.0,3.5,3.5,3.0,4.0,5.0,1.0,4.0,4.0,2.5,4.5,5.0,NaN,1.0,4.5,NaN,5.0,NaN,4.0,2.5,4.5,4.5,2.5,4.0,3.0,3.0,5.0,3.5,3.0,3.0,2.5,4.5,4.5,3.0,NaN,2.5,3.5,3.5,2.5,3.5,3.0,NaN,3.5,3.0,1.0,NaN,0.5,1.5,4.0,3.5,3.5,4.5,3.5,3.5,0.5,3.0,4.5,4.0,4.5,2.0,2.0,2.5,NaN,3.5,3.5,4.0
648,4.0,3.0,4.0,5.0,3.5,4.5,NaN,NaN,NaN,NaN,5.0,5.0,4.0,3.5,4.0,2.5,NaN,NaN,3.5,NaN,2.5,NaN,NaN,3.5,4.0,NaN,5.0,3.5,4.0,2.0,NaN,3.5,4.5,3.0,1.5,3.0,3.5,3.5,NaN,3.5,3.5,3.5,3.5,4.0,3.5,1.5,2.5,4.0,4.5,4.0,4.0,3.0,3.5,4.0,NaN,NaN,NaN,NaN,3.5,4.0,2.0,3.0,3.0,3.0,NaN,NaN,2.0,4.0,NaN,3.0,3.0,3.5,3.5,4.0,4.0,NaN
847,2.5,4.0,2.5,4.0,2.5,3.5,4.5,2.5,NaN,2.5,3.0,3.5,3.5,3.5,4.5,3.5,2.5,4.0,4.0,3.0,3.0,3.5,2.5,4.0,3.5,NaN,2.5,3.0,2.5,3.0,NaN,4.0,3.5,2.0,3.5,4.0,1.0,1.5,3.5,3.0,3.0,4.0,5.0,3.0,3.5,4.0,4.0,4.5,NaN,3.0,2.0,NaN,4.5,2.0,NaN,NaN,2.5,NaN,2.5,4.0,NaN,2.5,2.5,3.0,NaN,3.5,2.5,3.0,2.5,3.0,NaN,4.0,3.0,NaN,3.0,NaN
859,4.0,4.0,4.0,5.0,4.5,4.0,4.0,NaN,NaN,NaN,5.0,4.5,4.0,4.0,4.0,4.0,NaN,3.5,4.5,3.5,4.0,4.5,NaN,5.0,4.5,NaN,5.0,5.0,NaN,3.5,NaN,4.5,3.5,5.0,4.5,4.5,NaN,4.5,4.0,4.0,4.0,5.0,3.0,4.0,3.0,4.0,4.0,4.5,NaN,4.5,4.0,3.5,5.0,5.0,5.0,NaN,4.0,4.5,4.5,3.5,4.5,4.5,5.0,NaN,4.5,4.5,4.5,4.5,4.5,4.0,NaN,4.0,4.0,NaN,NaN,NaN
947,3.5,4.0,NaN,5.0,5.0,4.5,NaN,NaN,NaN,NaN,4.0,2.5,3.5,NaN,4.5,NaN,NaN,4.0,NaN,4.0,3.5,4.0,NaN,4.5,5.0,NaN,3.5,4.0,3.5,3.5,NaN,4.0,4.0,4.5,4.5,4.5,3.0,4.5,NaN,3.5,4.0,5.0,4.0,3.0,4.5,4.0,3.5,3.0,NaN,4.5,4.0,2.5,4.0,4.5,NaN,NaN,3.5,NaN,4.0,NaN,0.5,5.0,4.0,3.5,NaN,2.5,3.5,5.0,4.5,4.5,4.0,4.5,4.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161544,3.5,4.0,4.0,3.5,3.5,2.5,NaN,NaN,NaN,NaN,4.5,3.5,3.5,3.0,3.5,3.5,NaN,NaN,3.0,2.0,1.5,NaN,NaN,3.5,3.5,NaN,3.0,3.0,3.5,3.0,NaN,3.5,2.5,3.0,3.0,3.0,3.0,3.0,2.0,2.5,2.5,3.5,2.0,2.5,2.5,3.0,3.0,3.5,NaN,2.0,NaN,2.5,3.5,2.5,2.5,NaN,3.0,3.0,2.5,3.0,2.5,3.0,2.5,2.5,3.0,3.5,3.0,2.5,3.5,2.5,3.5,3.5,NaN,3.5,3.0,4.0
161586,3.5,4.5,4.5,3.0,2.5,3.5,4.0,NaN,3.0,NaN,3.5,4.0,3.5,4.0,3.0,3.5,1.5,4.0,3.0,1.5,2.5,NaN,2.0,2.5,3.5,NaN,3.5,3.5,2.0,2.0,NaN,NaN,3.0,3.0,3.5,3.0,2.5,3.0,2.0,3.0,1.5,3.0,4.0,3.0,2.0,2.5,NaN,NaN,5.0,3.0,2.5,NaN,3.0,3.0,NaN,2.0,NaN,NaN,3.0,4.0,3.5,3.0,4.0,3.5,NaN,NaN,4.0,3.0,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN
162047,5.0,5.0,3.5,5.0,3.5,4.0,4.0,NaN,NaN,3.5,5.0,3.0,5.0,3.5,4.0,3.5,NaN,4.5,4.0,3.5,3.5,3.0,4.0,2.5,4.5,NaN,4.5,3.5,3.5,4.0,NaN,3.0,4.5,4.5,2.5,5.0,5.0,3.5,4.5,4.0,4.0,4.5,4.0,4.0,4.0,4.0,4.5,5.0,NaN,4.5,4.0,5.0,4.0,4.5,3.0,NaN,3.0,4.0,4.0,4.5,4.5,4.0,5.0,4.0,4.0,4.0,4.5,4.5,3.5,4.0,4.5,5.0,4.0,3.0,3.5,4.5


Calculate correlations between selected user and other users.

In [22]:
corr_of_userIds_rated_same_movies_with_selected_userId = userIds_rated_atleast_X_perc_same_movies_with_selected_userId.T.corr().unstack()
corr_of_userIds_rated_same_movies_with_selected_userId

userId  userId
426     426       1.000000
        648       0.181479
        847      -0.002028
        859       0.056798
        947       0.077779
                    ...   
162516  161544   -0.041472
        161586    0.272322
        162047    0.020581
        162271   -0.033629
        162516    1.000000
Length: 1530169, dtype: float64

In [23]:
# converting correlation results to dataframe
userIds_corr_df = pd.DataFrame(corr_of_userIds_rated_same_movies_with_selected_userId, columns=["corr"])
userIds_corr_df.index.names = ["userId_1","userId_2"]
userIds_corr_df.reset_index(inplace=True)
userIds_corr_df

,userId_1,userId_2,corr
0,426,426,1.000000
1,426,648,0.181479
2,426,847,-0.002028
3,426,859,0.056798
4,426,947,0.077779
...,...,...,...
1530164,162516,161544,-0.041472
1530165,162516,161586,0.272322
1530166,162516,162047,0.020581
1530167,162516,162271,-0.033629


Leave only the user who has at least X percentage of correlation with selected user.

In [24]:
corr_threshold = 0.5 # users who have at least 60% correlation with selected user
final_users_corr_df = userIds_corr_df[(userIds_corr_df["userId_1"] == selected_userId) & 
                                      (userIds_corr_df["userId_2"] != selected_userId) &  # exclude selected user himself as his correlation with himself is 1
                                      (userIds_corr_df["corr"] >= corr_threshold)].sort_values(by="corr", ascending=False) # set threshold and sort correlations
final_users_corr_df

,userId_1,userId_2,corr
611475,63131,50996,0.561425
612304,63131,160951,0.546685
612253,63131,152788,0.534485
611987,63131,120901,0.531776
611181,63131,15753,0.520613
611724,63131,84658,0.519258
611988,63131,120986,0.516464
612174,63131,145061,0.508414
611708,63131,81896,0.501619


Filtering those users on 'User-Movie Matrix'.

In [25]:
list_users_to_filter = final_users_corr_df["userId_2"].to_list()

final_rec_df = user_movie_matrix.loc[list_users_to_filter,:]
final_rec_df # all movieIds and all users at least 70% rated and have at least 50% correlation with selected user

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,39,41,42,43,44,45,46,47,48,50,52,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,76,78,79,81,82,85,86,88,89,92,93,94,95,97,100,101,102,103,104,105,107,110,111,112,117,118,122,123,125,126,132,135,140,141,144,145,147,149,150,151,153,154,155,156,157,158,159,160,161,162,163,164,165,168,169,170,171,172,173,174,175,176,177,179,180,181,185,186,188,191,193,194,195,196,198,199,203,204,205,207,208,213,215,216,218,222,223,224,225,227,229,230,231,232,233,234,235,236,237,239,242,246,247,248,249,252,253,256,257,258,259,260,261,262,265,266,267,270,271,272,273,274,275,276,277,278,280,281,282,288,289,290,292,293,296,299,300,302,303,305,306,307,308,312,313,314,315,316,317,318,319,322,326,327,328,329,330,332,333,334,337,338,339,340,342,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,360,361,362,364,365,366,367,368,...,107141,107348,107406,107771,108188,108190,108727,108729,108932,108945,109374,109487,109578,109673,109848,110102,110127,110501,110553,110730,110882,111113,111360,111362,111364,111443,111622,111659,111743,111759,111781,111921,112138,112171,112175,112183,112290,112370,112552,112556,112623,112804,112852,112897,113345,113348,113378,113573,113741,114180,114662,114795,114935,115122,115149,115210,115569,115617,115713,116161,116797,116799,116823,116897,116977,117176,117444,117529,117533,117590,117881,117895,118696,118900,118997,119141,119145,119155,120466,120635,120799,121231,122882,122886,122890,122892,122896,122898,122900,122902,122904,122906,122910,122912,122914,122916,122918,122920,122922,122924,122926,125916,127098,127108,127202,128360,128838,129354,130073,130490,130520,130634,132046,132480,132796,133419,133771,134130,134170,134368,134393,134853,135133,135137,135143,135436,135536,135567,135569,135861,135887,136018,136020,136562,136864,137857,138036,139385,139644,140110,140174,140247,140267,140715,140956,141866,142448,142488,142997,143355,143385,143859,146656,148626,149334,149406,152017,152077,152081,152970,156387,157296,157699,158238,158783,158872,158966,159093,159415,159817,159858,160080,160271,160438,160718,160980,161131,161582,161634,162082,162350,162414,162578,162600,162606,163134,163645,164179,164909,165549,165551,166024,166461,166528,166534,166635,166643,166946,167746,168248,168250,168252,168254,168326,168366,168492,168612,169864,169984,170705,171011,171763,173145,173291,173941,174055,175303,175569,175661,176101,176371,176419,177593,177615,177763,177765,178061,179401,179817,179819,180031,182715,183611,183837,183869,183897,185029,187541,187593,187595,188301,189203,189333,189713,192385,192389,192803,194448,195159,201773
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
50996,2.0,2.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,4.5,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,NaN,NaN,4

As we want to recommend movies which are not rated/unseen by selected user, we should exclude the movies already watched and rated by selected user and the movies rated by no users.

In [26]:
final_rec_excluded_selected_user_movies_df = final_rec_df.T
final_rec_excluded_selected_user_movies_df = final_rec_excluded_selected_user_movies_df[~final_rec_excluded_selected_user_movies_df.index.isin(list_movieIds_ratedby_selected_userId)] # eliminating movies rated by selected user as we want to make any unseen movie
final_rec_excluded_selected_user_movies_df = final_rec_excluded_selected_user_movies_df.loc[~final_rec_excluded_selected_user_movies_df.apply(lambda row: row.isnull().all(), axis=1),:] # eliminating movies rated by none of users in the list
final_rec_excluded_selected_user_movies_df

userId,50996,160951,152788,120901,15753,84658,120986,145061,81896
movieId,,,,,,,,,
1,2.0,4.0,5.0,3.0,3.0,3.0,5.0,4.5,3.0
2,2.0,3.0,4.0,1.5,3.0,4.0,NaN,3.5,2.0
3,NaN,3.0,2.0,1.5,2.0,1.0,NaN,NaN,2.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
5,NaN,3.0,3.0,0.5,1.0,2.5,NaN,NaN,2.5
...,...,...,...,...,...,...,...,...,...
183897,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185029,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189203,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We adjust all users rating with correlations based on selected user.

In [27]:
final_rec_adjusted_df = (final_rec_excluded_selected_user_movies_df * np.array(final_users_corr_df["corr"].to_list()))
final_rec_adjusted_df

userId,50996,160951,152788,120901,15753,84658,120986,145061,81896
movieId,,,,,,,,,
1,1.122850,2.186739,2.672426,1.595327,1.561840,1.557775,2.582319,2.287862,1.504857
2,1.122850,1.640054,2.137941,0.797663,1.561840,2.077033,NaN,1.779448,1.003238
3,NaN,1.640054,1.068971,0.797663,1.041227,0.519258,NaN,NaN,1.003238
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.003238
5,NaN,1.640054,1.603456,0.265888,0.520613,1.298146,NaN,NaN,1.254047
...,...,...,...,...,...,...,...,...,...
183897,1.964987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185029,1.684275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189203,1.122850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now, we can get mean of all movie ratings not considering users as we standarized them using correlation based on selected user.

In [28]:
movie_scores_df = final_rec_adjusted_df.mean(axis=1).to_frame(name="RatingScore").reset_index().sort_values(by="RatingScore", ascending=False)
movie_scores_df

,movieId,RatingScore
1561,3365,2.672426
340,569,2.672426
1857,4322,2.672426
1497,3196,2.672426
3060,109848,2.658878
...,...,...
2386,8643,0.259629
1160,2399,0.259629
1710,3799,0.259629
453,881,0.259629


We can set a threshold for scores.

In [29]:
score_threshold = 3
movie_scores_df[movie_scores_df["RatingScore"] >= score_threshold]

,movieId,RatingScore


Finally, we can see the recommended movie names for selected user.

In [30]:
rec_movieIds = movie_scores_df["movieId"].to_list()
rec_scores = movie_scores_df["RatingScore"].to_list()
scores_count_to_show = 5

print("*"*100)
print("Recommendation List for User '", selected_userId, "'")
print("*"*100)
for no, movieId, rate in zip(range(1,scores_count_to_show+1),rec_movieIds[:scores_count_to_show], rec_scores[:scores_count_to_show]):
    movie_name = find_movie_title_using_id(movieId)
    score = round(rate,1)
    print(no,"-", movie_name, "-"*(70 - len(movie_name)-len(str(no))), " - ", score*2,"/ ", "(", "* "*round(score), ")")
print("*"*100)

###########################################################################################################################################

print("*"*100)
print("Movie List rated by User '", selected_userId, "'")
print("*"*100)
for no, movieId in enumerate(list_movieIds_ratedby_selected_userId,1):
    movie_name = find_movie_title_using_id(movieId)
    print(no,"-", movie_name, "-"*(70 - len(movie_name)-len(str(no))))
print("*"*100)

****************************************************************************************************
Recommendation List for User ' 63131 '
****************************************************************************************************
1 - Searchers, The (1956) ------------------------------------------------  -  5.4 /  ( * * *  )
2 - Little Big League (1994) ---------------------------------------------  -  5.4 /  ( * * *  )
3 - Eight Men Out (1988) -------------------------------------------------  -  5.4 /  ( * * *  )
4 - Stalag 17 (1953) -----------------------------------------------------  -  5.4 /  ( * * *  )
5 - Under the Skin (2013) ------------------------------------------------  -  5.4 /  ( * * *  )
****************************************************************************************************
****************************************************************************************************
Movie List rated by User ' 63131 '
*************************************

---
# Model Based Matrix Factorization
---

### _Data Preperation_

We need some additional libraries.

In [31]:
from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import train_test_split, GridSearchCV, cross_validate

We select random 10 movies from the first dataframe and create 'User-Movie-Matrix' based on selected movies

In [32]:
random_movieIds = [33794,2021,2571,5952,6323,44191,135887]
random_movies_df = df[df["movieId"].isin(random_movieIds)].sort_values(by="title")
umm_random_movies_df = random_movies_df.pivot_table(index="userId", columns="movieId", values="rating")
umm_random_movies_df

movieId,2021,2571,5952,6323,33794,44191,135887
userId,,,,,,,
1,NaN,NaN,4.0,NaN,NaN,NaN,NaN
2,NaN,5.0,5.0,NaN,5.0,NaN,NaN
3,4.0,4.0,4.0,4.0,4.0,4.0,NaN
4,NaN,4.5,4.5,NaN,3.5,2.5,NaN
13,NaN,4.5,4.5,NaN,4.5,4.5,NaN
...,...,...,...,...,...,...,...
162533,NaN,4.0,5.0,NaN,4.5,4.0,NaN
162534,NaN,4.5,3.0,NaN,4.0,NaN,NaN
162536,NaN,NaN,NaN,NaN,4.0,NaN,NaN


We define the scale for ratings using 'Reader' and create dataset in required format using 'Dataset' from surprise library

In [33]:
scale = Reader(rating_scale=(1,5))
mf_data = Dataset.load_from_df(random_movies_df[["userId","movieId","rating"]], scale)
mf_data

### _Modeling_

Splitting dataset into Train and Set and creating and fitting model

In [34]:
train_data, test_data = train_test_split(mf_data, test_size=0.2)
svd_model = SVD().fit(train_data)
svd_model

Making prediction and calculating RMSE

In [35]:
preds = svd_model.test(test_data)
print("Prediction Samples:")
display(preds[:10])

print("*"*100)
accuracy.rmse(preds)

Prediction Samples:


[Prediction(uid=145414, iid=2571, r_ui=4.0, est=4.02004011541878, details={'was_impossible': False}),
 Prediction(uid=161393, iid=33794, r_ui=5.0, est=4.278613924617449, details={'was_impossible': False}),
 Prediction(uid=109196, iid=2571, r_ui=5.0, est=4.190413589629608, details={'was_impossible': False}),
 Prediction(uid=114571, iid=2021, r_ui=5.0, est=3.339404721829171, details={'was_impossible': False}),
 Prediction(uid=46786, iid=5952, r_ui=5.0, est=4.374516575034455, details={'was_impossible': False}),
 Prediction(uid=34740, iid=33794, r_ui=2.0, est=3.7771702718315843, details={'was_impossible': False}),
 Prediction(uid=72315, iid=2021, r_ui=3.0, est=3.1191243878956283, details={'was_impossible': False}),
 Prediction(uid=69811, iid=44191, r_ui=4.5, est=4.022794306595468, details={'was_impossible': False}),
 Prediction(uid=89526, iid=2571, r_ui=5.0, est=4.2126295772894755, details={'was_impossible': False}),
 Prediction(uid=105819, iid=5952, r_ui=4.5, est=3.7330408209327204, detai

****************************************************************************************************
RMSE: 0.9250


0.9249613513247508

if we take as example the first line "Prediction(uid=125713, iid=5952, r_ui=4.0, est=4.114041949256712, details={'was_impossible': False})", we see that the user '125713' for the movie '5952' rated as '4.0' and the model estimated the rating as '4.11'.

### _Model Tuning_

We define a parameter set and find the best parametes using GridSearchCV

In [36]:
params = {"n_factors":[80,100],
          "n_epochs":[10,20],
          "lr_all":[0.002,0.005]}

svd_model_gs = GridSearchCV(SVD,
                            params,
                            cv=3,
                            measures=["rmse"],
                            n_jobs=-1,
                            joblib_verbose=True)

svd_model_gs.fit(mf_data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   29.0s finished


In [43]:
bs = svd_model_gs.best_score
bp = svd_model_gs.best_params["rmse"]

print("Best Score of CV:", bs["rmse"])
print("Best Parameters:")
bp

Best Score of CV: 0.9210195590878035
Best Parameters:


{'n_factors': 80, 'n_epochs': 20, 'lr_all': 0.005}

### _Final Model and Prediction_

In [47]:
mf_data_final = mf_data.build_full_trainset()
svd_mode_final = SVD(**bp)

svd_mode_final.fit(mf_data_final)

Finally, we can get the prediction for a user with found weights

In [48]:
svd_mode_final.predict(uid=145414, iid=2571)

Prediction(uid=145414, iid=2571, r_ui=None, est=4.077765046558609, details={'was_impossible': False})

---
# Preparation of Working Script
---

In [ ]:
def load_dataset():
    movies = pd.read_csv("data/movies.csv")
    ratings = pd.read_csv("data/ratings.csv")
    dataframe = ratings.merge(movies, how="left")
    return dataframe

def data_info(dataframe):
    print("Dataframe Shape:", dataframe.shape)
    print("-"*50)
    print("Dataframe Info:")
    display(dataframe.info())
    print("-"*50)
    print("Missing Values in Each Column:")
    display(dataframe.isnull().sum())
    print("-"*50)
    print("Dataframe Sample:")
    display(dataframe.head())

def data_stats(dataframe, user_col="user_id", item_col="item_id", user_rating_threshold=30, item_rated_threshold=1000):
    user_count = pd.DataFrame(dataframe[user_col].value_counts(), columns=["count"])
    product_count = pd.DataFrame(dataframe[item_col].value_counts(), columns=["count"])
    
    u_less_than_threshold = len(user_count[user_count["count"] < user_rating_threshold])
    u_more_than_threshold = len(user_count[user_count["count"] >= user_rating_threshold])
    
    m_less_than_threshold = len(product_count[product_count["count"] < item_rated_threshold])
    m_more_than_threshold = len(product_count[product_count["count"] >= item_rated_threshold])
    
    print("*"*100)
    print("Total User Count: ", len(user_count))
    print("User count rated less than", user_rating_threshold, "times: ", u_less_than_threshold,"(", round((u_less_than_threshold / len(user_count))*100,2), " %)")
    print("User count rated more than", user_rating_threshold, "times: ", u_more_than_threshold,"(", round((u_more_than_threshold / len(user_count))*100,2), " %)")
    print("-"*50)
    print("Total Product Count: ", len(product_count))
    print("Product count rated less than", item_rated_threshold, "times: ", m_less_than_threshold,"(", round((m_less_than_threshold / len(product_count))*100,2), " %)")
    print("Product count rated more than", item_rated_threshold, "times: ", m_more_than_threshold,"(", round((m_more_than_threshold / len(product_count))*100,2), " %)")
    print("*"*100)

def dataframe_reduction(dataframe, user_col="user_id", item_col="item_id", user_rating_threshold=30, item_rated_threshold=1000):
    user_count = pd.DataFrame(dataframe[user_col].value_counts(), columns=["count"])
    item_count = pd.DataFrame(dataframe[item_col].value_counts(), columns=["count"])
    
    items_to_be_removed = item_count[item_count["count"] < item_rated_threshold].index.to_list()
    users_to_be_removed = user_count[user_count["count"] < user_rating_threshold].index.to_list()
    
    reduced_dataframe = dataframe[~dataframe[item_col].isin(items_to_be_removed)]
    reduced_dataframe = reduced_dataframe[~reduced_dataframe[user_col].isin(users_to_be_removed)]
    
    before = len(user_count) * len(item_count)
    after = len(reduced_dataframe[user_col].value_counts()) * len(reduced_dataframe[item_col].value_counts())
    
    print("Matrix Size BEFORE reduction:", len(user_count), "*", len(item_count),"=", before)
    print("Matrix Size AFTER reduction:", len(reduced_dataframe[user_col].value_counts()), "*", len(reduced_dataframe[item_col].value_counts()),"=", after)
    print("Matrix Size Reduction Rate:", round((before - after)/before, 2)*100, "%")
    print("Information kept:", round(len(reduced_dataframe)/len(dataframe), 2)*100, "%")
    
    return reduced_dataframe



def find_item_name_using_id(dataframe, item_col_name="item_id", item_id=None):
    return dataframe[dataframe[item_col_name] == item_id]["item_name"].values[0]

def find_item_id_using_name(dataframe, item_col_name="item_name", item_name=None):
    return dataframe[dataframe[item_col_name] == item_name]["item_id"].values[0]
def search_item_names_with_keyword(dataframe, item_col_name="item_name", searched_item_name=None):
    item_names = list()
    [item_names.append(name) for name in dataframe[item_col_name] if searched_item_name.lower() in name.lower() and name not in item_names]
    return item_names
def create_user_item_matrix(dataframe, index_col="user_id", columns_col="item_id", values_col="rating"):
    user_item_matrix = dataframe.pivot_table(index=index_col, columns=columns_col, values=values_col)
    return user_item_matrix

def item_based_recommendation(user_item_matrix, dataframe, selected_item_id, top_n=10):
    selected_item = user_item_matrix.loc[:,selected_item_id]
    correlated_items = user_item_matrix.corrwith(selected_item).sort_values(ascending = False)[1:top_n+1] # [1:top_n+1] to exclude the item itself
    correlated_item_ids = correlated_items.index.to_list()
    correlated_rates = correlated_items.to_list()
    
    print("*"*100)
    print("'", find_item_name_using_id(dataframe, item_id=selected_item_id), "' - Recommendation List")
    print("*"*100)
    for no, item_id, rate in zip(range(1,top_n+1),correlated_item_ids, correlated_rates):
        item_name = find_item_name_using_id(dataframe, item_id=item_id)
        print(no,"-", item_name, "-"*(70 - len(item_name)-len(str(no))), " - ", round(rate*100,2),"%")
    print("*"*100)

def user_based_recommendation(user_item_matrix, dataframe, selected_user_id, perc_threshold_rated_same_products=0.7, corr_threshold=0.5, score_threshold=3, scores_count_to_show=5):
    umm_for_selected_user = user_item_matrix[user_item_matrix.index == selected_user_id]
    bool_for_selected_user = umm_for_selected_user.apply(lambda col: col.notnull(), axis=1)
    
    item_ids_ratedby_selected_user = bool_for_selected_user.iloc[0].loc[lambda item_id: item_id == True]
    list_item_ids_ratedby_selected_user = bool_for_selected_user.iloc[0].loc[lambda item_id: item_id == True].index.to_list()
    
    allUserIds_and_items_ratedby_selected_user = user_item_matrix.loc[:, list_item_ids_ratedby_selected_user]
    
    count_threshold_rated_same_items = len(list_item_ids_ratedby_selected_user) * perc_threshold_rated_same_products
    userIds_rated_atleast_X_perc_same_items_with_selected_user = allUserIds_and_items_ratedby_selected_user[allUserIds_and_items_ratedby_selected_user.notnull().sum(axis=1) > count_threshold_rated_same_items]
    
    corr_of_userIds_rated_same_items_with_selected_user = userIds_rated_atleast_X_perc_same_items_with_selected_user.T.corr().unstack()
    
    # converting correlation results to dataframe
    userIds_corr_df = pd.DataFrame(corr_of_userIds_rated_same_items_with_selected_user, columns=["corr"])
    userIds_corr_df.index.names = ["userId_1","userId_2"]
    userIds_corr_df.reset_index(inplace=True)
    
    final_users_corr_df = userIds_corr_df[(userIds_corr_df["userId_1"] == selected_user_id) & 
                                          (userIds_corr_df["userId_2"] != selected_user_id) &  # exclude selected user himself as his correlation with himself is 1
                                          (userIds_corr_df["corr"] >= corr_threshold)].sort_values(by="corr", ascending=False) # set threshold and sort correlations
    
    list_users_to_filter = final_users_corr_df["userId_2"].to_list()
    
    final_rec_df = user_item_matrix.loc[list_users_to_filter,:]
    
    final_rec_excluded_selected_user_items_df = final_rec_df.T
    final_rec_excluded_selected_user_items_df = final_rec_excluded_selected_user_items_df[~final_rec_excluded_selected_user_items_df.index.isin(list_item_ids_ratedby_selected_user)] # eliminating items rated by selected user as we want to make any unseen item
    final_rec_excluded_selected_user_items_df = final_rec_excluded_selected_user_items_df.loc[~final_rec_excluded_selected_user_items_df.apply(lambda row: row.isnull().all(), axis=1),:] # eliminating items rated by none of users in the list
    return final_rec_excluded_selected_user_items_df

def model_based_matrix_factorization(dataframe, index_col="user_id", columns_col="item_id", values_col="rating"):
    scale = Reader(rating_scale=(dataframe[values_col].min(),dataframe[values_col].max()))
    mf_data = Dataset.load_from_df(dataframe[[index_col,columns_col,values_col]], scale)
    
    mf_data_final = mf_data.build_full_trainset()
    svd_model_final = SVD()
    
    svd_model_final.fit(mf_data_final)
    
    return svd_model_final                                                                   









---
## I hope you found the analysis insightful and informative!

## Your feedback is greatly appreciated, and I welcome any suggestions for improvement. Feel free to reach out with any questions or comments. 👍🏼
---